In [ ]:
# Only for Colab, comment out if not using Colab
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Change this line if you're using Colab to something like '/content/drive/MyDrive/TeamX/'
# where TeamX is just the clone of repository on your Google Drive
# and you have mounted the drive at /content/drive  
# See the Tutorial Slides for more detail.

# Works on your local machine but not on Colab!
PROJECT_ROOT = '/icbb/projects/igunduz/hlcv-project' 

# Fix this path and use this one on Colab
# PROJECT_ROOT = '/content/drive/MyDrive/TeamX' 

In [ ]:
import sys
from os.path import join as ospj

sys.path.append(ospj(PROJECT_ROOT, 'src'))

%load_ext autoreload
%autoreload 2

## Playing around with the data
### 1. Load the data
Using the IIT-AFF dataset, we will load the data and see what it looks like.
### 2. Visualize the data
We will visualize the data to see what it looks like.
### 3. Preprocess the data
Create a DataLoaders object for the data.

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as T
import cv2


data_dir = ospj(PROJECT_ROOT, 'data', 'IIT_Affordances_2017', 'IIT_Affordances_2017')

background = (200, 222, 250)
c1 = (0,0,205)   
c2 = (34,139,34) 
c3 = (192,192,128)   
c4 = (165,42,42)    
c5 = (128,64,128)   
c6 = (204,102,0)  
c7 = (184,134,11) 
c8 = (0,153,153)
c9 = (0,134,141)
c10 = (184,0,141) 
c11 = (184,134,0) 
c12 = (184,134,223)
label_colours = np.array([background, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12])

# Object
col0 = (0, 0, 0)
col1 = (0, 255, 255)
col2 = (255, 0, 255)
col3 = (0, 125, 255)
col4 = (55, 125, 0)
col5 = (255, 50, 75)
col6 = (100, 100, 50)
col7 = (25, 234, 54)
col8 = (156, 65, 15)
col9 = (215, 25, 155)
col10 = (25, 25, 155)

col_map = [col0, col1, col2, col3, col4, col5, col6, col7, col8, col9, col10]


# img = Image.open(ospj(data_dir, 'rgb', '00_00000090.jpg'))
# plt.imshow(img)

In [ ]:
import math
print(math.sqrt(562500))

## Converting Text File to Segmentation Map

In [ ]:
affordance_path = ospj(data_dir, 'affordances_labels', 'ILSVRC2014_train_00044663.txt')

with open(affordance_path) as f:
    affordance_labels = f.readlines()
    dim1 = len(affordance_labels)
    dim2 = len(affordance_labels[0].split())
    
count = 0

# Creation of Affordance pixel grid
# Checking image size to continue with creating Segmentation Map
affordance_pixels = []
for label in affordance_labels:
    row_pixels = []
    digits = label.split()
    count = 0
    for digit in digits:
        if digit.isdigit():
            row_pixels.append(digit)
            count += 1
    assert count==dim2, f"Actual count: {count}"
    affordance_pixels.append(row_pixels)
        
print(count)

dimensions = [dim1, dim2]
print(dimensions)

affordance_pixels = np.array(affordance_pixels, dtype=np.uint8)
print(len(affordance_pixels))

seg_map = label_colours[affordance_pixels]
print(seg_map.shape)

seg_map = np.array(seg_map, dtype=np.uint8)
print(seg_map.shape)

# bgr_image = cv2.cvtColor(seg_map, cv2.COLOR_RGB2BGR)

plt.imshow(seg_map)
plt.show()


In [ ]:
img_141 = cv2.imread(ospj(data_dir, 'rgb', 'ILSVRC2014_train_00044663.jpg'))
print(f"Image shape: {img_141.shape}")
print(f"Mask shape: {seg_map.shape}")
plt.imshow(img_141)
plt.show()

In [ ]:
from transformers import SegformerFeatureExtractor, SegformerForImageClassification
from PIL import Image
import requests

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

seg_b0_feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/mit-b0")
seg_b0_encoder_model = SegformerForImageClassification.from_pretrained("nvidia/mit-b0")

# inputs = seg_b0_feature_extractor(images=img_141, segmentation_maps=seg_map, return_tensors="pt")

# print(inputs.keys())

# outputs = seg_b0_encoder_model(**inputs)
# logits = outputs.logits
# # model predicts one of the 1000 ImageNet classes
# predicted_class_idx = logits.argmax(-1).item()
# print("Predicted class:", model.config.id2label[predicted_class_idx])


In [ ]:
print(seg_b0_encoder_model)

In [ ]:
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
from PIL import Image
import requests

seg_b0_processor = SegformerImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
seg_b0_model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

print(seg_b0_model)


In [ ]:
from PIL import Image

img = Image.open(ospj(data_dir, 'rgb', 'ILSVRC2014_train_00044663.jpg'))
plt.imshow(img)

In [ ]:
inputs = processor_1(images=img_141, bgr_image, return_tensors="pt")
outputs = model_1(**inputs)
logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)

In [ ]:
import numpy as np

plt.imshow(np.mean(logits.squeeze().detach().numpy(), axis=0))

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt


data_dir = ospj(PROJECT_ROOT, 'data', 'VOCdevkit2012', 'VOC2012', 'JPEGImages')
img = Image.open(ospj(data_dir, '1347.jpg'))
plt.imshow(img)

In [ ]:
from transformers import YolosImageProcessor, YolosForObjectDetection
from PIL import Image
import torch
import requests

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny')
image_processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")



In [ ]:
inputs = image_processor(images=img, return_tensors="pt")
outputs = model(**inputs)

# model predicts bounding boxes and corresponding COCO classes
logits = outputs.logits
bboxes = outputs.pred_boxes


# print results
target_sizes = torch.tensor([img.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.6, target_sizes=target_sizes)[0]
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )


In [ ]:
print(results)

In [ ]:
from data_loaders.AffordanceDataset import AffordanceDataset, collate_fn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor, Normalize, Compose, Resize



transform = Compose([Resize((600,600)),
                     ToTensor()])

# transform = None

train_dataset = AffordanceDataset(root_dir=data_dir,
                                split_file="train_and_val.txt",
                                feature_extractor=seg_b0_feature_extractor,
                                transform=transform)

validation_dataset = AffordanceDataset(root_dir=data_dir, 
                                  split_file="val.txt",
                                  feature_extractor=seg_b0_feature_extractor, 
                                  transform=transform)

batch_size = 8


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, collate_fn=collate_fn)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, num_workers=2, collate_fn=collate_fn)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
for batch in validation_loader:
    input = batch['image']
    target = batch['affordances_labels']
    print(input.shape)
    print(target.shape)